# Badminton Wind Predictor - Quick Start

This notebook demonstrates the complete workflow:
1. Generate synthetic weather data
2. Preprocess and build features
3. Train a tiny LSTM model (1 epoch for speed)
4. Make forecasts and get play/don't play decision

**Runtime:** ~2-3 minutes on Colab free tier

## Setup

Clone the repo and install dependencies:

In [ ]:
# Clone repository (skip if already cloned)
!git clone https://github.com/yourusername/badminton-wind-predictor.git
%cd badminton-wind-predictor

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

## 1. Generate Sample Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.config import SAMPLE_DATA_PATH
from scripts.make_sample_data import generate_synthetic_weather_data

# Generate 2000 hours of synthetic data
df = generate_synthetic_weather_data(n_records=2000, seed=42)

# Save to file
df.to_csv(SAMPLE_DATA_PATH, index=False)

print(f"Generated {len(df)} hours of data")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")

# Display sample
df.head()

In [ ]:
# Visualize wind data
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

# Wind speed
axes[0].plot(df['timestamp'], df['wind_m_s'], label='Wind Speed', alpha=0.7)
axes[0].plot(df['timestamp'], df['wind_gust_m_s'], label='Wind Gusts', alpha=0.7)
axes[0].axhline(y=1.5, color='r', linestyle='--', label='Play threshold (median)')
axes[0].set_ylabel('Wind Speed (m/s)')
axes[0].set_title('Synthetic Wind Data')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Pressure
axes[1].plot(df['timestamp'], df['pressure'], color='green')
axes[1].set_ylabel('Pressure (hPa)')
axes[1].set_xlabel('Timestamp')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Preprocess Data

In [ ]:
from src.data.fetch import load_sample
from src.data.preprocess import build_features

# Load data
df_raw = load_sample()

print(f"Raw data shape: {df_raw.shape}")

# Build features
df_processed = build_features(df_raw)

print(f"Processed data shape: {df_processed.shape}")
print(f"\nFeature columns: {df_processed.columns.tolist()[:10]}...")

# Show sample
df_processed.head()

## 3. Train LSTM Model (Quick)

Training for just 1 epoch to demonstrate the pipeline (in practice, use 50+ epochs).

In [ ]:
from src.models.lstm_model import LSTMForecaster

# Split data
split_idx = int(len(df_processed) * 0.8)
train_df = df_processed.iloc[:split_idx]
val_df = df_processed.iloc[split_idx:]

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}")

# Create and train model
model = LSTMForecaster()

print("\nTraining LSTM for 1 epoch (demo mode)...")
model.fit(train_df, val_df, epochs=1)

print("\n✓ Training complete!")

## 4. Make Forecast and Decision

In [ ]:
from src.decision.rules import decide_play

# Get latest forecast
median_forecast = model.predict_latest(df_processed)

# Compute q90 (simplified)
q90_forecast = {key: value * 1.2 for key, value in median_forecast.items()}

print("Median Forecast:")
for horizon, value in median_forecast.items():
    print(f"  {horizon}: {value:.2f} m/s")

print("\nQ90 Forecast (90th percentile):")
for horizon, value in q90_forecast.items():
    print(f"  {horizon}: {value:.2f} m/s")

In [ ]:
# Make decision
decision = decide_play(median_forecast, q90_forecast)

print("="*60)
print(f"DECISION: {decision['decision']}")
print("="*60)
print(f"\nReason: {decision['reason']}")
print(f"\nThresholds:")
for key, value in decision['thresholds'].items():
    print(f"  {key}: {value}")

print(f"\nPer-Horizon Details:")
for horizon, details in decision['details'].items():
    status = "✓ PASS" if details['passes'] else "✗ FAIL"
    print(f"  {horizon}: {status}")
    print(f"    Median: {details['median_wind_m_s']:.2f} m/s")
    print(f"    Q90: {details['q90_wind_m_s']:.2f} m/s")
    if details['violations']:
        print(f"    Violations: {details['violations']}")

## 5. Visualize Forecast

In [ ]:
# Extract horizon values
horizons = [1, 3, 6]
median_values = [median_forecast[f'horizon_{h}h'] for h in horizons]
q90_values = [q90_forecast[f'horizon_{h}h'] for h in horizons]

# Create visualization
fig, ax = plt.subplots(figsize=(10, 6))

x = np.array(horizons)
ax.plot(x, median_values, marker='o', linewidth=2, markersize=8, label='Median Forecast')
ax.fill_between(x, median_values, q90_values, alpha=0.3, label='Uncertainty (50-90%)')

# Threshold lines
ax.axhline(y=1.5, color='orange', linestyle='--', linewidth=2, label='Median Threshold')
ax.axhline(y=2.5, color='red', linestyle='--', linewidth=2, label='Q90 Threshold')

ax.set_xlabel('Forecast Horizon (hours)', fontsize=12)
ax.set_ylabel('Wind Speed (m/s)', fontsize=12)
ax.set_title('Wind Forecast with Uncertainty', fontsize=14, fontweight='bold')
ax.set_xticks(horizons)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

# Add decision text
decision_color = 'green' if decision['decision'] == 'PLAY' else 'red'
ax.text(0.98, 0.98, f"{decision['decision']}", 
        transform=ax.transAxes,
        fontsize=16, fontweight='bold',
        verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor=decision_color, alpha=0.3))

plt.tight_layout()
plt.show()

## Summary

You've successfully:

1. ✓ Generated synthetic weather data
2. ✓ Preprocessed with feature engineering
3. ✓ Trained an LSTM forecasting model
4. ✓ Made multi-horizon wind forecasts
5. ✓ Got a play/don't play decision

### Next Steps

- Train for more epochs (50+) for better accuracy
- Try different model architectures
- Use real weather data (METAR, OpenWeather)
- Deploy to Hugging Face Spaces
- Customize thresholds for your preferences

### Learn More

- Documentation: See `README.md` and `docs/design.md`
- Source code: Explore `src/` directory
- Tests: Check `tests/` for examples